In [ ]:
import cv2
import matplotlib.pyplot as plt

# video = cv2.VideoCapture('videos/plane.mp4')
video = cv2.VideoCapture('videos/crowd.mp4')
ret, frame = video.read()

# Ручной выбор объекта
bbox = cv2.selectROI("select_bbox", frame, fromCenter=False, showCrosshair=True)
cv2.destroyWindow("select_bbox")
print(bbox)

In [ ]:
# Список видео с границами объекта
VIDEO_SAMPLES = [
    {'file_path': 'videos/birds.mp4', 'bbox': (816, 208, 49, 20)},
    {'file_path': 'videos/plane.mp4', 'bbox': (1192, 453, 250, 207)},
    {'file_path': 'videos/crowd.mp4', 'bbox': (623, 153, 25, 53)},
    {'file_path': 'videos/football.mp4', 'bbox': (326, 37, 18, 62)},
]

# Список трекеров 
TRACKERS = {
    'MIL': (cv2.TrackerMIL.create, (255, 0, 0)),
    'KCF': (cv2.legacy.TrackerKCF_create, (0, 255, 0)),
    'CSRT': (cv2.legacy.TrackerCSRT_create, (0, 0, 255)),
}

In [ ]:
def process_video(tracker_id: str, video_path: str, bbox: tuple):
    '''
    Метод обрабатывает видео (используя переданные трекер) и добавляет рамку для выделенного объекта.
    Результат сохраняется в другом видеофайле
    
    :param tracker_id: идентификатор трекера
    :param video_path: путь к видео файлу
    :param bbox: начальные границы объекта на видео
    :return: 
    '''
    video = cv2.VideoCapture(video_path)
    
    # Настройка параметров для output видео
    fps = int(video.get(cv2.CAP_PROP_FPS))
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    
    output_video = f"videos/{video_path.split('/')[-1].split('.')[0]}_out_{tracker_id}.mp4"
    out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))
    
    # инициализация трекера
    ret, frame = video.read()
    tracker = TRACKERS[tracker_id][0]()
    colour = TRACKERS[tracker_id][1]    
    tracker.init(frame, bbox)
    
    while True:
        ret, frame = video.read()
        if not ret:
            break
        
        is_success, bbox = tracker.update(frame)
        if is_success:
            x, y, w, h = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), colour, 2)
            cv2.putText(frame, tracker_id, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour, 2)
            plt.imshow(frame)
    
        out.write(frame)

    video.release()
    out.release()
    return output_video


In [ ]:
# обработка видео для трех трекеров MIL, CSRT, KCF
for video_sample in VIDEO_SAMPLES:
    video_path = video_sample['file_path']
    bbox = video_sample['bbox']
    process_video('MIL', video_path, bbox)
    process_video('CSRT', video_path, bbox)
    process_video('KCF', video_path, bbox)

### ИТОГИ
####
В этой работе сравнивалась работа трех трекеров MIL, CSRT и KCF. При этом использовались разные видео (по качеству, динамике) и отслеживаемые объекты- ( с изменением и без изменения формы, размера; частично заслоненные и скрывающиеся). Анализируя полученные результы можно сделать следующие выводы:

**1.** Качество отслеживания зависит от ряда факторов:
  - качества видео и динамики сцен
  - изначально определенной границы объекта
  - используемого трекера

**2.** Из всех трекеров **лучшее качество** трекинга показал **CSRT**. Однако время обработки файла самое большое

**3.** Быстрее всех работает **KCF**. Однако очень чевствителен к изменяющемуся фону или сильной деформации объекта. Часто объекты терялись из виду.

**4.** Трекер **MIL** отработал в ситуациях, когда объект временно пропадает из кадра.

Трекер CSRT был самым стабильным, но медленным, а KCF – быстрым, но менее универсальным.

Видео результаты в каталоге [lesson_10/videos](https://github.com/chaplinskij/rd_computer_vision/tree/main/lesson_10/videos)
[![Results](https://github.com/chaplinskij/rd_computer_vision/blob/main/lesson_10/image.png?raw=true)](https://github.com/chaplinskij/rd_computer_vision/blob/main/lesson_10/videos/result.MD "Results")